## Load sampled images and download detailed metadata

In [1]:
import sys
sys.path.append("D:\\projects\\geo_image_experiments\\src")
import importlib
import mapillary_utils as mu
importlib.reload(mu)

<module 'mapillary_utils' from 'D:\\projects\\geo_image_experiments\\src\\mapillary_utils.py'>

In [2]:
import requests
from dotenv import load_dotenv
from os import getenv

from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np

import sqlite3
import json

import requests


pd.set_option('display.max_colwidth', None)

In [3]:
test_dir = "d://projects_working_directories//202408_pano_images"
file = "360_images_2024-09-02.db"

images_dir = f"{test_dir}//images"

db_file = f"{test_dir}//{file}"

load_dotenv()
API_KEY = getenv("MAPILLARY_CLIENT_TOKEN")

In [4]:
# load sampled images 
conn = sqlite3.connect(f"{test_dir}//{file}")
df_sampled_images = pd.read_sql('select * from sampled_images', conn)
conn.close()

In [5]:
df_sampled_images

,image_lat,image_lon,residual,image_id,camera_type,is_pano,camera_focal_len,camera_k1,camera_k2,image_path,error,image_url,altitude,compass_angle,exif_orientation,make,model,country,city
0,-6.263354,106.784390,177.236655,1505018570392136,spherical,1,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An_BsALpAV6f4DjjTTJtnYEmWGn0qyiZBfqTLBnouxJf8qVF3EXPd4t_dkoFEDG2Yh70iVl5ffChtFrDOEdpPfQK0HnHdO_7pcciHxWJF1NhFRk2YxMIsvOt_8xq2WFpYOar0McXKsL_A4gYBiG-ag?ccb=10-5&oh=00_AYBN0sl2UXdDmOtKX2Vqnq3hIo5ASmzhY3ZOXsv9xyhUsQ&oe=66FDBD62&_nc_sid=201bca,48.755,262.83362725926,1,GoPro,GoPro Max,Indonesia,Jakarta
1,-7.426615,112.750108,140.153000,1095545924696245,spherical,1,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An9BXK5mtDvCgjbmUTy1wTsPT-jZ52uqemltQp7fSb1M1ShfnsyEuNrmec4hqZrK6J4q4e7u8zH-tpDIMGYQJxyIjVlZIn90bjRyYF1Tkiy9-_cvGzZWewrWw4UN1yxZw86z4_qAXTQKJkVVthCURWc?ccb=10-5&oh=00_AYDCJxd1xmk-ndCM_iZELDbkYcJ0s0bMzge9ksxHDa8eeA&oe=66FDA6B9&_nc_sid=201bca,32.769,0,1,GoPro,GoPro Max,Indonesia,Surabaya
2,-7.273224,112.638557,217.079601,1564726257278323,spherical,1,None,None,None,None,None,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An9C6SQRBJOSyDTV4EtkMPP8Lfacn7bVtbFXUUsO0p-BdB9GskemlmlLksdA_Ac5bq5SP0qQTCpvoVOGbpuajaLNEcza_qv6wYhItBPDJa4UD5Hlq4CzFKIVmOPR3s3_9PfDIxWy472faxLRfbbpWg?ccb=10-5&oh=00_AYClVYLKBogwyqaxRSd7qKx6w0o67yzfN7Xau9KOC9ntgg&oe=66FD8D29&_nc_sid=201bca,58.031,0,1,GoPro,GoPro Max,Indonesia,Surabaya


In [6]:
#image_id = '856814222158020'

#image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator='thumb_2048_url', image_dir=images_dir)
#print("Metadata:", metadata.keys)

In [7]:
image_size_cycle = [
    #'thumb_original_url','thumb_2048_url'
    'thumb_2048_url'
]

for image_id in df_sampled_images['image_id'].tolist():
    for image_size in image_size_cycle:
        image, metadata = mu.get_mapillary_image(image_id, API_KEY, image_size_indicator=image_size, image_dir=images_dir)
        try:
            df_temp = pd.DataFrame.from_dict([metadata])
            df_metadata = pd.concat([df_metadata,df_temp], ignore_index=True)
        except:
            df_metadata = pd.DataFrame.from_dict([metadata])

{'type': 'Point', 'coordinates': [106.7843904, -6.2633539]}
{'type': 'Point', 'coordinates': [112.75010779997, -7.4266151]}
{'type': 'Point', 'coordinates': [112.6385565, -7.2732235]}


In [8]:
def is_image_original_size(row):
    if (row.original_height == row.height) & (row.original_width == row.width):
        return 1
    else:
        return 0

df_metadata['is_image_original_size'] = df_metadata.apply(is_image_original_size, axis=1)

In [9]:
conn = sqlite3.connect(db_file)

df_metadata.to_sql('image_metadata', con=conn, if_exists='replace', index=False)



# Close the connection
conn.close()

## Add Exif Data Back

In [10]:
from PIL import Image
import piexif

In [11]:
df_metadata = df_metadata.merge(df_sampled_images[['altitude','make','model','image_id']], how='left', left_on='image_id', right_on='image_id')

In [12]:
df_metadata

,guid,image_source,image_id,captured_at_unix,lat,lon,original_height,original_width,height,width,...,compass_angle,is_pano,camera_focal_len,camera_k1,camera_k2,image_path_on_disk,is_image_original_size,altitude,make,model
0,649730a1a36845518d5dc5757e348a1f,mapillary,1505018570392136,1696075183000,-6.263354,106.784390,2880,5760,1024,2048,...,262.833627,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1505018570392136_thumb_2048_url.jpg,0,48.755,GoPro,GoPro Max
1,051a3e7a00c74e0284605f0273b3557c,mapillary,1095545924696245,1658328383000,-7.426615,112.750108,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1095545924696245_thumb_2048_url.jpg,0,32.769,GoPro,GoPro Max
2,cda51a1a400f4b9789981f12f9908121,mapillary,1564726257278323,1658593288000,-7.273224,112.638557,2880,5760,1024,2048,...,0.000000,True,None,None,None,d://projects_working_directories//202408_pano_images//images//mapillary_1564726257278323_thumb_2048_url.jpg,0,58.031,GoPro,GoPro Max


In [13]:
from geopy import Point
from geopy.distance import distance

# Decimal to DMS
def decimal_to_dms(lat, lon):
    point = Point(latitude=lat, longitude=lon)
    
    
    lat_dms = point.format().split(',')[0]
    lon_dms = point.format().split(',')[1]
    
    return lat_dms, lon_dms


lat, lon = -6.222932, 106.833616
lat_dms, lon_dms = decimal_to_dms(lat, lon)
print(f"Latitude: {lat_dms}, Longitude: {lon_dms}")

Latitude: 6 13m 22.5552s S, Longitude:  106 50m 1.0176s E


In [14]:


from datetime import datetime

def unix_to_exif_datetime(unix_timestamp):
    # Convert Unix timestamp to a datetime object
    dt = datetime.utcfromtimestamp(unix_timestamp)
    
    # Format datetime as "YYYY:MM:DD HH:MM:SS"
    exif_datetime = dt.strftime("%Y:%m:%d %H:%M:%S")
    
    return exif_datetime
    
unix_to_exif_datetime(1000 / 1000)


'1970:01:01 00:00:01'

In [15]:
def build_exif_0th(make='', model=''):
    
    zero_th =  {
        piexif.ImageIFD.Make: make,
        piexif.ImageIFD.Model: model
    }

    return zero_th

def build_exif_exif(datetime=1000, lensmake=None, lensmodel=None):
    
    
    exif = {
        piexif.ExifIFD.DateTimeOriginal: unix_to_exif_datetime(datetime / 1000),
        piexif.ExifIFD.LensMake: "",
        piexif.ExifIFD.LensModel: ""
    }

    return exif

print(build_exif_0th(make='GoPro', model='360 Max'))
print(build_exif_exif(datetime=1000))


{271: 'GoPro', 272: '360 Max'}
{36867: '1970:01:01 00:00:01', 42035: '', 42036: ''}


In [16]:
def build_exif_gps(lat, lon):
    # pass in lat/lon in decimal format
    # then convert it to a string
    # then convert to exif format
    # 
    # can probably convert directly but
    # I don't know how
    

    
    def split_elements(element):
        def to_rational(number):
            # Convert a number to a rational tuple (numerator, denominator)
            return (int(number * 1000), 1000)
    
    
        degrees, minutes, seconds, reference = element.split()
        degrees = abs(int(degrees))
        minutes = int(minutes[0:-1])
        seconds = to_rational(float(seconds[0:-1]))

        return ((degrees,1),(minutes,1),seconds) , reference

    
    point = Point(lat, lon)
    string_point = point.format()
    string_lat, string_lon = string_point.split(', ')
    
    print(string_point)
    
    lat_elements, lat_ref = split_elements(string_lat)
    lon_elements, lon_ref = split_elements(string_lon)

    gps_ifd = {
        piexif.GPSIFD.GPSLatitudeRef: lat_ref.encode(),
        piexif.GPSIFD.GPSLatitude: lat_elements,
        piexif.GPSIFD.GPSLongitudeRef: lon_ref.encode(),
        piexif.GPSIFD.GPSLongitude: lon_elements,
    }

    return gps_ifd

build_exif_gps(37.7749, -122.4194)

37 46m 29.64s N, 122 25m 9.84s W


{1: b'N',
 2: ((37, 1), (46, 1), (29640, 1000)),
 3: b'W',
 4: ((122, 1), (25, 1), (9840, 1000))}

In [17]:
def generate_exif_data(row):
    
    zero_th = build_exif_0th(make=row.make, model=row.model)
    exif = build_exif_exif(datetime=row.captured_at_unix)
    gps = build_exif_gps(row.lat, row.lon)

    print(zero_th)

    exif_data = {
        "0th" : zero_th,
        "Exif" : exif,
        "GPS" : gps

    }

    return exif_data

df_metadata['exif_dict'] = df_metadata.apply(generate_exif_data, axis=1)

6 15m 48.074s S, 106 47m 3.80544s E
{271: 'GoPro', 272: 'GoPro Max'}
7 25m 35.8144s S, 112 45m 0.38808s E
{271: 'GoPro', 272: 'GoPro Max'}
7 16m 23.6046s S, 112 38m 18.8034s E
{271: 'GoPro', 272: 'GoPro Max'}


In [23]:
df_metadata = df_metadata.merge(df_sampled_images, how='left', left_on='image_id', right_on='image_id')
df_metadata.to_excel("c://temp//foo.xlsx")

In [19]:
def add_exif(image_path, exif_data):

    # Open the image
    image = Image.open(image_path)

    output_path = image_path + "with_exif.jpg"

    # Extract existing EXIF data (if any)
    existing_exif = image.info.get('exif')


    # Load existing EXIF data
    exif_dict = piexif.load(existing_exif) if existing_exif else {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}, "thumbnail": None}
    print(exif_dict)

    # Modify or add new EXIF tags
    for ifd in ("0th", "Exif", "GPS", "1st"):
        if ifd in exif_data:
            for tag, value in exif_data[ifd].items():
                print(f"{tag} - {value} - {ifd}")
                exif_dict[ifd][tag] = value

    # Convert the modified EXIF data back to binary
    exif_bytes = piexif.dump(exif_dict)

    # Save the image with the new EXIF data
    print(output_path)
    image.save(output_path, exif=exif_bytes)


exif_data = {
    "0th": {
        piexif.ImageIFD.Make: "Camera Brand",
        piexif.ImageIFD.Model: "Camera Model",
        piexif.ImageIFD.Software: "My Software"
    },
    "Exif": {
        piexif.ExifIFD.DateTimeOriginal: "2024:08:31 12:34:56",
        piexif.ExifIFD.LensMake: "Lens Brand",
        piexif.ExifIFD.LensModel: "Lens Model"
    },
    "GPS": {
        piexif.GPSIFD.GPSLatitudeRef: "N",
        piexif.GPSIFD.GPSLatitude: ((40, 1), (0, 1), (0, 100)),  # 40 degrees
        piexif.GPSIFD.GPSLongitudeRef: "W",
        piexif.GPSIFD.GPSLongitude: ((74, 1), (0, 1), (0, 100)),  # 74 degrees
    }
}

#add_exif('d://projects_working_directories//202408_pano_images//images//mapillary_1332341057452601_thumb_2048_url.jpg', exif_data)

In [20]:
df_metadata.apply(lambda x: add_exif(x.image_path_on_disk, x.exif_dict), axis=1)

{'0th': {}, 'Exif': {}, 'GPS': {}, '1st': {}, 'thumbnail': None}
271 - GoPro - 0th
272 - GoPro Max - 0th
36867 - 2023:09:30 11:59:43 - Exif
42035 -  - Exif
42036 -  - Exif
1 - b'S' - GPS
2 - ((6, 1), (15, 1), (48074, 1000)) - GPS
3 - b'E' - GPS
4 - ((106, 1), (47, 1), (3805, 1000)) - GPS
d://projects_working_directories//202408_pano_images//images//mapillary_1505018570392136_thumb_2048_url.jpgwith_exif.jpg
{'0th': {}, 'Exif': {}, 'GPS': {}, '1st': {}, 'thumbnail': None}
271 - GoPro - 0th
272 - GoPro Max - 0th
36867 - 2022:07:20 14:46:23 - Exif
42035 -  - Exif
42036 -  - Exif
1 - b'S' - GPS
2 - ((7, 1), (25, 1), (35814, 1000)) - GPS
3 - b'E' - GPS
4 - ((112, 1), (45, 1), (388, 1000)) - GPS
d://projects_working_directories//202408_pano_images//images//mapillary_1095545924696245_thumb_2048_url.jpgwith_exif.jpg
{'0th': {}, 'Exif': {}, 'GPS': {}, '1st': {}, 'thumbnail': None}
271 - GoPro - 0th
272 - GoPro Max - 0th
36867 - 2022:07:23 16:21:28 - Exif
42035 -  - Exif
42036 -  - Exif
1 - b'S' 

0    None
1    None
2    None
dtype: object